In [12]:
#"/global/u1/z/zhwang/zhong/lib/python2.7/site-packages",
#"/global/u1/z/zhwang/zhong/lib/python2.7/site-packages/IPython/extensions"
import sys, os

from pandas import DataFrame

add_libs = [
"D:\Anaconda\Lib\site-packages",
"D:\Anaconda\Lib\site-packages\IPython\extensions"
]
[sys.path.append(l) for l in add_libs]
import re
import numpy as np
import pandas as pd  #将pandas命名为pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
plt.rcParams['figure.figsize'] = (12, 8)
%config InlineBackend.figure_format='retina'
#定义图形细节

In [13]:
def get_pacbio_key(keyfile, inputfile):
    
    ## read annotated transcripts  读取注释转录本
    keyinfo = {}
    if not(os.path.exists(keyfile) and os.path.exists(inputfile)):
        print ("at least one of the input files are not found.")
        sys.exit(0)
    with open(keyfile, 'r') as KEY:
        for lines in KEY:  #.readlines():
            try:
                anarray = lines.strip("\n").split("\t")  #strip：删除字符串中的\n,split:将字符串使用\t分割
                keyinfo[anarray[0]] = anarray[4]
            except:
                continue
    KEY.close()
    #输出注释中的顶点数
    print ("Number of reads in the annotation: " + '{:d}'.format(len(keyinfo.keys())))   #keys:keys() 方法返回 view 对象。这个视图对象包含列表形式的字典键。
    known_clusters: DataFrame = pd.DataFrame.from_dict(keyinfo, 'index')  #将生成的图 生成一个字典
    #known_counts = known_clusters.groupby([0]).size()  #通过字典中的第0个元素排序，并计算字典中的数量
    known_counts = known_clusters.groupby(level = 0).size()  #通过字典中的第0个元素排序，并计算字典中的数量
    #在已经注释的簇中的总reads数量
    print ("Total reads in annotated clusters: " + '{:d}'.format(known_counts[known_counts>1].sum()))
    print ("Total annotated clusters: " + '{:d}'.format(sum(known_counts>1)))
    #已经注释的簇的总数
    
    ## annotate input reads---inputfile是seq文件
    annotations = {}
    no_input_reads = 0
    no_annotated_reads = 0
    with open(inputfile, 'r') as IN:  #以只读方式打开inputfile
        for lines in IN:#.readlines():
            seq_id,seq = lines.strip("\n").split("\t") #删除每一行的\n,通过\t分割，\t之前的赋值给seq_id,\t之后的赋值给seq
            header = seq_id.split(" ")[0] #通过“ ”分割，然后取第0个值赋值给header
            try:
                tid = keyinfo[header]
                no_annotated_reads += 1
            except:
                tid = '-1'
            annotations[seq_id] = tid    
            no_input_reads +=1
    #统计输入read总数
    print ("Number of reads in the input: " + '{:d}'.format(no_input_reads))
    print ("Number of reads annotated in the input: " + '{:d}'.format(no_annotated_reads))
    #统计输入的已经有注释的reads
    return annotations

def annotate_clusters(cluster, annotations):
    ''' 
    parse spark cluster results  解析spark聚类结果
    cluster file format is: seqid \t cluster_id  cluster文件格式：seqid  clusterid
    return: [seq_name, cluster_id, annotation_source, annotation_transcript, annotation_transcript_variant]
    
    ''' 
    if not os.path.exists(cluster):
        print ("Cluster file not found.")
        sys.exit(0)
        
    results = []
    total_clustered_reads = 0
    total_clustered_annotated = 0
    with open(cluster, 'r') as IN:
        for lines in IN:#.readlines():
            seq_id, group_id = lines.strip("\n").split("\t")
            total_clustered_reads += 1
            header = seq_id.split(" ")[0]
            if group_id > 0 and annotations[seq_id] != '-1':
                results.append([header, group_id] + annotations[seq_id].split('.'))
                total_clustered_annotated += 1
                
    IN.close()

    print ("Total reads in clusters: " + '{:d}'.format(total_clustered_reads))
    print ("Total annotated reads in clusters: " + '{:d}'.format(total_clustered_annotated))
    return pd.DataFrame(results) 


In [14]:
def clustering_stat(config):
    '''
    [seq_name, cluster_id, annotation_source, annotation_transcript, annotation_transcript_variant]
    '''
    ## purity，纯度
    grouped = config[config[3] >0].groupby([1])
    cluster_results = []
    for name,group in grouped:
        counts = group.groupby([3]).count()[0]
        cluster_results.append([ len(counts), sum(counts), max(counts), int(counts[counts == max(counts)].index[0])])
    cluster_results = np.array(cluster_results)
    clustered_reads = sum(cluster_results[:,1])
    print ("Total clusters: " + '{:d}'.format(cluster_results.shape[0]))
    print ('{:d}'.format(clustered_reads) + " reads are in clusters")
    print ("The largest cluster has " + '{:d}'.format(max(cluster_results[:,1]))  + " reads.")
    print ("Percent of 100% pure clusters: " + '{:.2f}'.format(100.0* sum(cluster_results[:,0] == 1)/cluster_results.shape[0]))
    print ("Median purity: " + '{:.2f}'.format(np.median(100.0* cluster_results[:,2] / cluster_results[:,1])))
    print ("Mean purity: " + '{:.2f}'.format(np.mean(100.0* cluster_results[:,2] / cluster_results[:,1])))

    # completeness  完整性
    grouped = config[config[3] >0].groupby([3])
    completeness = []
    for name,group in grouped:
        if len(group) <= 2:
            continue
        counts = group.groupby([1]).count()[0]
        completeness.append([max(counts), sum(counts)])
    completeness = np.array(completeness)    
    median_completeness = np.median(100.0 * completeness[:,0] / completeness[:,1])
    mean_completeness = np.mean(100.0 * completeness[:,0] / completeness[:,1])
    
    print ("The most abundant transcript has " + '{:d}'.format(max(completeness[:,1]))  + " copies.")
    print ("Median completeness: " + '{:.2f}'.format(median_completeness)  )
    print ("Mean completeness: " + '{:.2f}'.format(mean_completeness) )
    
    fig1 = plt.figure(num=1, figsize=(13, 12), dpi=80, facecolor='w', edgecolor='k')
    plt.subplots_adjust( wspace=.2, hspace=.2 )

    ax1 = fig1.add_subplot(3,3,1)
    ax1.set_title('Clusters Size Distribution (log10) vs purity')
    x = np.log10(cluster_results[:,1])
    y = 100.0* cluster_results[:,2] / cluster_results[:,1]
    sns.kdeplot(x, y, n_levels=100, shade=True, shade_lowest=True, ax=ax1);

    ax2 = fig1.add_subplot(3,3,2)
    ax2.set_title('Number of Transcripts(log2) per Cluster Distribution')
    sns.distplot(np.log2(cluster_results[:,0]), kde=False, bins=100, ax=ax2)
    
    ax3 = fig1.add_subplot(3,3,3)
    ax3.set_title('#Transcripts (log10) vs completeness')
    x = np.log10(completeness[:,1])
    y = 100.0 * completeness[:,0] / completeness[:,1]
    sns.kdeplot(x, y, n_levels=100, shade=True, shade_lowest=True, ax=ax3);

    # large clusters  大型集群
    large = 1
    x = np.log10(cluster_results[:,1])
    sel = x>=large
    if np.sum(sel)>10:
        ax4 = fig1.add_subplot(3,3,4)
        ax4.set_title('Large Clusters Size(log10) vs purity')
        y = 100.0* cluster_results[:,2] / cluster_results[:,1]
        sns.kdeplot(x[sel], y[sel], n_levels=10, shade=True, shade_lowest=True, ax=ax4);
        ax4.set_xlim(left=0)

        ax5 = fig1.add_subplot(3,3,5)
        ax5.set_title('Number of Transcripts(log2) per Large Clusters')
        ax5.set_ylim(top=2000)
        sns.distplot(np.log2(cluster_results[:,0]), kde=False, ax=ax5)

        ax6 = fig1.add_subplot(3,3,6)
        ax6.set_title('#Transcripts (log10) vs completeness for abundant transcripts')
        x = np.log10(completeness[:,1])
        sel = x>=large
        y = 100.0 * completeness[:,0] / completeness[:,1]
        sns.kdeplot(x[sel], y[sel], n_levels=10, shade=True, shade_lowest=True, ax=ax6);
        ax6.set_xlim(left=0)
 
    # largest clusters  最大的集群
    large = 2
    x = np.log10(cluster_results[:,1])
    sel = x>=large 
    if np.sum(sel)>1:
        ax7 = fig1.add_subplot(3,3,7)
        ax7.set_title('Largest Clusters Size(log10) vs purity')
        y = 100.0* cluster_results[:,2] / cluster_results[:,1]
        sns.regplot(x=x[sel], y=y[sel], ax=ax7);
        ax7.set_xlim(left=0)
        ax7.set_ylim(top=100)

        ax8 = fig1.add_subplot(3,3,8)
        ax8.set_title('Number of Transcripts(log2) per Largest Clusters')
        ax8.set_ylim(top=200)
        sns.distplot(np.log2(cluster_results[:,0]), kde=False, ax=ax8)

        ax9 = fig1.add_subplot(3,3,9)
        ax9.set_title('#Transcripts (log10) vs completeness for top transcripts')
        x = np.log10(completeness[:,1])
        sel = x>=large
        y = 100.0 * completeness[:,0] / completeness[:,1]
        sns.regplot(x=x[sel], y=y[sel], ax=ax9);
        ax9.set_xlim(left=0)
    
    plt.tight_layout()
    plt.show()

In [9]:
inputfile = 'D:\Study\SAPRC\temp\bovine\bovine.seq'
keyfile = "D:\Study\SAPRC\temp\bovine\addseq.txt"
#inputfile = '../pacbio/all_samples.seq'
#keyfile = "../pacbio/IsoSeq_Alzheimer_2016edition_polished.promiscuous.unimapped.read_stat.txt"
keyinfo=get_pacbio_key(keyfile, inputfile)


at least one of the input files are not found.


SystemExit: 0

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
'''
31-kmer reads graph, min 20 shared kmers between two reads
'''
config1 = annotate_clusters('../pacbio/pacbio4G_lpaseq.txt_31.min20', keyinfo)
clustering_stat(config1)

In [ ]:
'''
31-kmer reads graph, min 40 shared kmers between two reads
'''
config1 = annotate_clusters('../pacbio/pacbio4G_lpaseq.txt_31.min40', keyinfo)
clustering_stat(config1)

In [ ]:
'''
31-kmer reads graph, min 60 shared kmers between two reads
'''
config1 = annotate_clusters('../pacbio/pacbio4G_lpaseq.txt_31.min60', keyinfo)
clustering_stat(config1)